In [3]:
from transformers import pipeline
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType


StatementMeta(, d03d0294-6123-4376-b706-658c096dd173, 6, Finished, Available, Finished)

In [4]:

# 1️⃣ Load products
df = spark.read.table("GoldLakehouse.dbo.Dim_Products")

# 2️⃣ Initialize Hugging Face pipeline
sentiment_model = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# 3️⃣ Define UDF for inference
@udf(returnType=StringType())
def analyze_sentiment(text):
    try:
        return sentiment_model(text[:512])[0]['label']   # truncate long text
    except Exception:
        return "NEUTRAL"


StatementMeta(, d03d0294-6123-4376-b706-658c096dd173, 7, Finished, Available, Finished)

In [7]:
# 4️⃣ Apply model on description/reviews
df_with_sentiment = df.withColumn("Sentiment", analyze_sentiment(col("Description")))


# 6️⃣ Save enriched Gold table
df_with_sentiment.write.format("delta").mode("overwrite").saveAsTable("Gold_Product_Sentiment")


StatementMeta(, d03d0294-6123-4376-b706-658c096dd173, 10, Finished, Available, Finished)